In [1]:
using DataStructures
using Distributions

mutable struct TabuState{TMove,P,TF}
    tabu_buffer::CircularBuffer{TMove}
    best_seen::P
    best_seen_obj::TF
    current::P
    considered::P
    iter::Int
end

function TabuState(p, x0; buffer_length::Int=10)
    moves = possible_moves(p, x0)
    obj = objective(p, x0)
    return TabuState{eltype(moves),typeof(x0),typeof(obj)}(
        CircularBuffer{eltype(moves)}(buffer_length), x0, obj, copy(x0), copy(x0), 1
    )
end


function solve_tabu(p, s::TabuState; iteration_limit::Int=100)
    while s.iter < iteration_limit
        moves = possible_moves(p, s.current)
        best_move = 0
        best_move_obj = Inf
        for (i_move, move) in enumerate(moves)
            if in(move, s.tabu_buffer)
                # move forbidden, do not consider
                continue
            end
            # evaluate move
            copyto!(s.considered, s.current)
            apply!(s.considered, move)
            considered_value = objective(p, s.considered)
            if considered_value < best_move_obj
                best_move = i_move
                best_move_obj = considered_value
            end
        end
        # no allowed move found
        if best_move == 0
            break
        end
        apply!(s.current, moves[best_move])
        push!(s.tabu_buffer, invert_move(p, moves[best_move]))
        if best_move_obj < s.best_seen_obj
            # best so far, let's remember it
            copyto!(s.best_seen, s.current)
            s.best_seen_obj = best_move_obj
        end
        s.iter += 1
    end
    return s.best_seen
end


struct KnapsackProblem
    capacity::Int
    weights::Vector{Int}
    profits::Vector{Int}
end

function objective(p::KnapsackProblem, x)
    return -sum(p.profits .* x)
end


function apply!(x, move::Tuple{Symbol,Int})
    if move[1] === :add
        x[move[2]] = true
    else
        x[move[2]] = false
    end
    return x
end

function invert_move(::KnapsackProblem, move::Tuple{Symbol,Int})
    if move[1] === :add
        return (:remove, move[2])
    else
        return (:add, move[2])
    end
end


function possible_moves(p::KnapsackProblem, x::Vector{Bool})
    move_list = Tuple{Symbol,Int}[]
    current_weight = sum(p.weights .* x)
    # add item
    for i in eachindex(x, p.weights)
        if !x[i] && current_weight + p.weights[i] <= p.capacity
            push!(move_list, (:add, i))
        end
    end
    # remove item
    for i in eachindex(x, p.weights)
        if x[i]
            push!(move_list, (:remove, i))
        end
    end
    return move_list
end



possible_moves (generic function with 1 method)

In [10]:

function generate_problem()
    n_items = 100
    profits = rand(DiscreteUniform(10, 1000), n_items)
    weights = rand(DiscreteUniform(10, 100), n_items)
    kp = KnapsackProblem(3000, profits, weights)
end

kp1 = generate_problem()


KnapsackProblem(3000, [433, 122, 602, 499, 507, 250, 300, 531, 882, 498  …  695, 855, 445, 525, 57, 826, 587, 102, 590, 762], [70, 21, 56, 95, 92, 16, 23, 50, 92, 34  …  52, 40, 16, 82, 99, 70, 94, 62, 26, 10])

In [18]:

function test(kp)
    x0 = fill(false, length(kp.weights))
    st = TabuState(kp, x0; buffer_length=10)
    sol = solve_tabu(kp, st; iteration_limit=1000000)
    println(findall(sol))
    println("Best objective: ", st.best_seen_obj)
    println("Last iteration: ", st.iter)
end

test(kp1)

[22, 31, 32, 45, 48, 54, 56, 57, 60, 64, 69, 70, 78, 79, 81, 95, 98]
Best objective: -1431
Last iteration: 1000000
